### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [ ]:
#installation step
!pip install transformers
!pip install sentencepiece
!pip install utils
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab/
!mkdir data/model/
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

In [6]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob, time

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, T5Model


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split

from utils import *
from custom_field import *

%load_ext autoreload

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
VOCAB_SIZE = 5000

In [3]:
ls data/AD_NMT-master

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [4]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [5]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [ ]:
ls data/AD_NMT-master/

In [ ]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2, random_state=22)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2, random_state=22)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2, random_state=22)
mag_msa_test = data_Magribi_MSA_test

In [ ]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

In [ ]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [ ]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [ ]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [ ]:
#spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))
#spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt --model_prefix=data/model/eng --vocab_size=' + str(VOCAB_SIZE))
#spm.SentencePieceTrainer.train('--input=data/vocab/msa_vocab.txt --model_prefix=data/model/msa --vocab_size=' + str(VOCAB_SIZE))
#spm.SentencePieceTrainer.train('--input=data/vocab/lav_vocab.txt --model_prefix=data/model/lav --vocab_size=' + str(VOCAB_SIZE))
#spm.SentencePieceTrainer.train('--input=data/vocab/mag_vocab.txt --model_prefix=data/model/mag --vocab_size=' + str(VOCAB_SIZE))

In [7]:
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))

True

In [8]:
ls data/model

eng.model  lav.model  mag.model  msa.model  spm.model
eng.vocab  lav.vocab  mag.vocab  msa.vocab  spm.vocab


In [13]:
sp = spm.SentencePieceProcessor()
sp.load('data/model/spm.model')

True

In [14]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[1256, 3, 123, 3, 64, 3, 49, 2328]


In [15]:
print(sp.encode_as_pieces('هناك شوكة ناقصة'))
print(sp.encode_as_ids('هناك شوكة ناقصة'))

['▁هناك', '▁شو', 'ك', 'ة', '▁', 'نا', 'قصة']
[818, 931, 11, 12, 3, 23, 1167]


In [16]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[1256, 3, 123, 3, 64, 3, 49, 2328]


## Spacy Tokenizer

In [17]:
eng_vocab_data = open("data/model/eng.vocab", "r")
msa_vocab_data = open("data/model/msa.vocab", "r")
lav_vocab_data = open("data/model/lav.vocab", "r")
mag_vocab_data = open("data/model/mag.vocab", "r")
spm_vocab_data = open("data/model/spm.vocab", "r")

eng_vocab_list = []
msa_vocab_list = []
lav_vocab_list = []
mag_vocab_list = []
spm_vocab_list = []

for line in eng_vocab_data.readlines():
    eng_vocab_list.append(line.split("\t")[0])

for line in msa_vocab_data.readlines():
    msa_vocab_list.append(line.split("\t")[0])

for line in lav_vocab_data.readlines():
    lav_vocab_list.append(line.split("\t")[0])

for line in mag_vocab_data.readlines():
    mag_vocab_list.append(line.split("\t")[0])
    
for line in spm_vocab_data.readlines():
    spm_vocab_list.append(line.split("\t")[0])


In [18]:
spm_vocab_list[0:200]

['<unk>',
 '<s>',
 '</s>',
 '▁',
 'ا',
 'و',
 'ي',
 'ت',
 'ال',
 'ب',
 'م',
 'ك',
 'ة',
 'ه',
 'ن',
 'ل',
 'في',
 'ها',
 'من',
 'ما',
 'ش',
 'ان',
 'ف',
 'نا',
 '؟',
 'ين',
 'وا',
 'ون',
 'لي',
 'لا',
 'س',
 'ع',
 'على',
 "'",
 'ني',
 's',
 'انا',
 'هم',
 '▁و',
 'ى',
 'هو',
 'ح',
 'د',
 'انت',
 'ق',
 'يا',
 'ات',
 '▁لا',
 'فيال',
 't',
 'هذا',
 '▁انا',
 'الم',
 '?',
 '▁نعم',
 'مع',
 'ر',
 'ز',
 'عن',
 'بال',
 'سي',
 '▁I',
 'هي',
 '▁ا',
 'a',
 'كان',
 'ج',
 'خ',
 'ص',
 'انه',
 'كل',
 'منال',
 'تي',
 'غ',
 'لل',
 'لك',
 'ط',
 'ية',
 'حتى',
 'to',
 'سا',
 'بس',
 'او',
 'ذلك',
 'بي',
 'ض',
 'اني',
 'm',
 'شي',
 'اليوم',
 'اذا',
 'ار',
 'الى',
 'واحد',
 'ست',
 'لو',
 'هناك',
 'بت',
 'اء',
 'كنت',
 'كم',
 'وال',
 'غير',
 'ور',
 'ولا',
 'اللي',
 'مت',
 '▁اه',
 'لما',
 'the',
 'you',
 'بعد',
 'كي',
 'الله',
 'هنا',
 'هل',
 '▁هل',
 'عندما',
 'يت',
 'اد',
 'الان',
 '▁ايه',
 'd',
 'is',
 'جدا',
 'ing',
 'كانت',
 'مش',
 'لكن',
 'كيف',
 'ام',
 'علىال',
 'r',
 'فى',
 'ذ',
 'الا',
 'تو',
 'سليمة',
 '

In [19]:
en_vocab = Vocab(strings=eng_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

msa_vocab = Vocab(strings=msa_vocab_list)
spacy_msa_tokenizer = Tokenizer(msa_vocab)

lav_vocab = Vocab(strings=lav_vocab_list)
spacy_lav_tokenizer = Tokenizer(lav_vocab)

mag_vocab = Vocab(strings=mag_vocab_list)
spacy_mag_tokenizer = Tokenizer(mag_vocab)

spm_vocab = Vocab(strings=spm_vocab_list)
spacy_spm_tokenizer = Tokenizer(spm_vocab)

In [32]:
spacy_spm_tokenizer("james is testing ")

james is testing 

In [33]:
callable(spacy_spm_tokenizer)

True

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [ ]:
ls data/model

In [37]:
tokenizer = T5Tokenizer('data/model/spm.model')
print(callable(T5Tokenizer))

True


In [ ]:
input_ids = tokenizer.encode('تعلمت كيفية ركوب الدراجة عندما كان عمري ست سنوات ', return_tensors='pt')

In [ ]:
input_ids

In [ ]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?
output = tokenizer.decode(input_ids[0])
print(output)

In [ ]:
#from transformers import T5WithLMHeadModel
#model = T5WithLMHeadModel.from_pretrained('t5-small')
input_ids = tokenizer.encode('translate English to Arabic: She hates green peppers', return_tensor='pt')
lm_labels = tokenizer.encode('انها لا تحب الفلفل الاخضر', return_tensor='pt')

print(input_ids)
print(lm_labels)


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [38]:
SRC = Field(tokenize = spacy_spm_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_spm_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

SRC = Field(tokenize =  T5Tokenizer,
             init_token = '<sos>',
             eos_token = '<eos>',
             lower = False)

TRG = Field(tokenize =  T5Tokenizer,
             init_token = '<sos>',
             eos_token = '<eos>',
             lower = False)

In [74]:
SRC = Custom_Field(tokenize = "spacy",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Custom_Field(tokenize = "spacy",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

used spacy spm tokenizer
used spacy spm tokenizer


In [80]:
SRC.use_vocab

True

In [76]:
tok = SRC.tokenize

In [77]:
output = tok("james is a dog")
print(type(output))
print(output)

<class 'spacy.tokens.doc.Doc'>
james is a dog


In [81]:
eng_msa_train_dataset = TranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'), fields=(SRC, TRG))
lav_msa_train_dataset = TranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'), fields=(SRC, TRG))
mag_msa_train_dataset = TranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'), fields=(SRC, TRG))

In [82]:
SRC.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)

In [83]:
SRC.vocab

In [ ]:
"""SRC.build_vocab(eng_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset)
SRC.build_vocab(lav_msa_train_dataset)
TRG.build_vocab(lav_msa_train_dataset)
SRC.build_vocab(mag_msa_train_dataset)
TRG.build_vocab(mag_msa_train_dataset)"""

In [86]:
ex = eng_msa_train_dataset[0]

In [87]:
print(ex.src) 
print(ex.trg)

TRAIN_000\01\I don't remember what happened anymore
TRAIN_000\01\لم اعد اتذكر ما حصل


In [ ]:
ls data/train

In [88]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'),
                                                    fields = (SRC, TRG))

In [92]:
dir(train_data[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'fromCSV',
 'fromJSON',
 'fromdict',
 'fromlist',
 'fromtree',
 'src',
 'trg']

In [94]:
dir(train_data.src)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [95]:
train_data.src

<generator object Dataset.__getattr__ at 0x1a3eef30d0>

In [89]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

In [90]:
count = 0

for x in train_iterator:
    print(x.src)
    print(x.trg)
    #print(spacy_en_tokenizer.decode(x.src))
    count += 1
    
    if count > 3:
        break
    
print(count)

tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 3, 0, 0],
        [0, 0, 0, 3, 3, 3, 0, 1, 3, 0, 0, 3, 0, 0, 0, 1, 1, 0, 3, 1, 3, 3, 1, 3,
         3, 0, 0, 0, 0, 1, 0, 0],
        [0, 3, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 3, 1, 1, 0, 1, 1, 1, 1, 1, 1,
         1, 3, 0, 0, 0, 1, 3, 0],
        [3, 1, 0, 1, 1, 1, 3, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [66]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [67]:
def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [68]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing?

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [69]:
#declare model
config = T5Config(vocab_size=VOCAB_SIZE)
model = T5Model(config)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

In [71]:
config

T5Config {
  "_num_labels": 2,
  "architectures": null,
  "bad_words_ids": null,
  "bos_token_id": null,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout_rate": 0.1,
  "early_stopping": false,
  "eos_token_id": 1,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_factor": 1.0,
  "is_decoder": false,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 1.0,
  "max_length": 20,
  "min_length": 0,
  "model_type": "t5",
  "n_positions": 512,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_heads": 8,
  "num_layers": 6,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "prefix": null,
  "pruned_heads": {},
  "relative_attention_num_buckets": 32,
  "repetition_penalty": 1.0,
  "task_specific_params": n

In [70]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

RuntimeError: The size of tensor a (18) must match the size of tensor b (17) at non-singleton dimension 0

In [ ]:
def trainIters(model, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    optimizer = AdamW.SGD(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX) #CHECK IF THE BEST LOSS FUNCTION

    for iter in range(1, n_iters + 1):
        

        loss = train(decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
model = 
n_iteres = 10
trainIters(model, n_iters)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [ ]:
from tqdm import tqdm
def generate_translations(lns, output_file_path, model_size, batch_size, device):
    model = T5ForConditionalGeneration.from_pretrained(model_size)
    model.to(device)
    #returned to using T5 tokenizer
    tokenizer = tokenizer

    task_specific_params = model.config.task_specific_params

    with Path(output_file_path).open("w") as output_file:
        for batch in tqdm(list(chunks(lns, batch_size))):
        #modify to have prefix 'translate English to Arabic: ...'
            batch = [ text for text in batch]
            
            dct = tokenizer.batch_encode_plus(batch, max_length=512, return_tensors="pt", pad_to_max_length=True)

            input_ids = dct["input_ids"].to(device)
            attention_mask = dct["attention_mask"].to(device)

            translations = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            dec = [
                tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in translations
            ]

            for hypothesis in dec:
                output_file.write(hypothesis + "\n")

In [ ]:
from pathlib import Path
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    with Path(score_path).open("w") as score_file:
        score_file.write(result)

In [ ]:
!mkdir data/output/
!touch data/output/translated.msa
!touch data/output/score_eng_msa.txt

In [ ]:
#got rid of the argparse and just put it in as params
def run_generate(model_size='t5-base', input_path='data/train/train_eng_msa.eng', output_path='data/output/translated.msa', ref_path='data/train/train_eng_msa.eng', score_path='data/output/score_eng_msa.txt', batch_size=BATCH_SIZE):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # Read input lines into python
    with open(input_path, "r") as input_file:
        input_lns = [x.strip() for x in input_file.readlines()]

    generate_translations(input_lns, output_path, model_size, batch_size, device)

    # Read generated lines into python
    with open(output_path, "r") as output_file:
        output_lns = [x.strip() for x in output_file.readlines()]

    # Read reference lines into python
    with open(ref_path, "r") as reference_file:
        refs_lns = [x.strip() for x in reference_file.readlines()]

    calculate_bleu_score(output_lns, refs_lns, args.score_path)

In [ ]:
run_generate()